In [ ]:
from tensorflow.keras.applications import DenseNet121
from data_proccessing import image_size, labels
import tensorflow as tf

dnet = DenseNet121(
        input_shape=(image_size, image_size, 3),
        weights='imagenet',
        include_top=False
    )
dnet.trainable = True

dnet_model = tf.keras.Sequential([
    dnet,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(labels), activation='sigmoid')])

dnet_model.compile(
    optimizer='adam',
    loss = 'binary_crossentropy',
    metrics=['AUC','Precision','Recall'])

In [ ]:
from data_proccessing import train_gen, valid_gen, X_val, y_val

history = dnet_model.fit(train_gen,
              validation_data = valid_gen,
              epochs = 10,)

y_pred = dnet_model.predict(X_val)

In [ ]:
import numpy as np
for label, p_count, t_count in zip(labels,
                                     100 * np.mean(y_pred, 0),
                                     100 * np.mean(y_val, 0)):
    print('%s: actual: %2.2f%%, predicted: %2.2f%%' % (label, t_count, p_count))